In [2]:
import requests 
import pandas as pd 
import numpy as np
from dateutil import parser 

## Accéder à l'identifiant d'un film : cas pour un film 

In [36]:
key_api='d1d1413d8379729633d60e9f5cc4a730' #clé de l'API 

In [37]:
film='The Fabelmans'
url=f"https://api.themoviedb.org/3/search/movie?api_key={key_api}&query={film}" 

#requête pour trouver les infos  
req = requests.get(url)
carte = req.json()

ind=[]
for film in range(len(carte['results'])):
    ind.append(carte['results'][film]['id'])
print(ind)  

[804095, 1201609]


## Généralisation pour plusieurs films 

#### fonction pour recuperer l'id des films 

In [38]:
def id_recup(titre):
    url_api=f"https://api.themoviedb.org/3/search/movie?api_key={key_api}&query={titre}" 
    req = requests.get(url_api)
    carte = req.json()

    ind=[]
    for film in range(len(carte['results'])):
        ind.append(carte['results'][film]['id'])
    return(ind)

#### fonction pour récuperer les infos du film à partir de l'id 

In [39]:
def df_avec_id(id):
    id_film= id
    url_new_api = f"https://api.themoviedb.org/3/movie/{id_film}?api_key={key_api}&language=en-US"
    req_new = requests.get(url_new_api)
    wb_new = req_new.json()
    
    
    #ajustement des données 
    if 'belongs_to_collection' in wb_new and wb_new['belongs_to_collection'] is not None:
        wb_new['belongs_to_collection'] = wb_new['belongs_to_collection']['name']

        
    # Ajustement des données pour la clé 'genres'
    if 'genres' in wb_new:
        wb_new['genres'] = ', '.join([x['name'] for x in wb_new['genres']])
    else:
        wb_new['genres'] = None

# Ajustement des données pour la clé 'production_companies'
    if 'production_companies' in wb_new:
        wb_new['production_companies'] = ', '.join([x['name'] for x in wb_new['production_companies']])
    else:
        wb_new['production_companies'] = None

# Ajustement des données pour la clé 'production_countries'
    if 'production_countries' in wb_new:
        wb_new['production_countries'] = ', '.join([x['name'] for x in wb_new['production_countries']])
    else:
        wb_new['production_countries'] = None

# Ajustement des données pour la clé 'spoken_languages'
    if 'spoken_languages' in wb_new:
        wb_new['spoken_languages'] = ', '.join([x['name'] for x in wb_new['spoken_languages']])
    else:
        wb_new['spoken_languages'] = None

    
    df=pd.DataFrame(wb_new, index=[0])
    
    return (df)

#### Création de la database 

In [40]:
def get_movie_info(movie_id_list):
    # Initialiser un DataFrame vide pour stocker les informations sur les films
    movie_df = pd.DataFrame()

    for movie_id in movie_id_list:
        # Utiliser la deuxième fonction pour obtenir les informations détaillées du film
        movie_info = df_avec_id(movie_id)

        # Vérifier si des informations ont été trouvées
        if movie_info is not None:
            # Ajouter les informations du film au DataFrame
            movie_df = pd.concat([movie_df, movie_info], ignore_index=True)

    return movie_df

#Exemple 
#movie_id_list = get_id("Inception") 
#result_df = get_movie_info(movie_id_list)
#result_df.head()

In [41]:
def create_movie_list(movie_list):
    all_movies_df = pd.DataFrame()

    for movie_name in movie_list:
        movie_id_list = id_recup(movie_name)
        if movie_id_list:
            movie_info_df = get_movie_info(movie_id_list)
            # Ajouter les informations du film au DataFrame global
            all_movies_df = pd.concat([all_movies_df, movie_info_df], ignore_index=True)

    return all_movies_df

# Exemple 
#movie_list = ["Inception", "The Dark Knight", "Interstellar"]
#result_df = create_movie_list_dataframe(movie_list)
#result_df.head()

#### recuperer les titres des films sous forme de liste 

In [42]:
df_movies= pd.read_csv("df_film_avec_critiques.csv")
liste_des_films = df_movies['Titre original'].tolist()
#print(liste_des_films)
len(liste_des_films)

4194

In [43]:
moitie = len(liste_des_films) // 2

# Séparation de la liste en deux parties
liste_partie_1 = liste_des_films[:moitie]
liste_partie_2 = liste_des_films[moitie:]

moit = len(liste_partie_1)//2
half = len(liste_partie_2)//2

quart= liste_partie_1[:moit]
deux_quart= liste_partie_1[moit:]
trois_quart= liste_partie_2[:half]
quatre_quart= liste_partie_2[half:]


# Affichage des deux listes résultantes
print("Partie 1:", len(quart))
print("Partie 2:", len(deux_quart))
print("Partie 3:", len(trois_quart))
print("Partie 4:", len(quatre_quart))

Partie 1: 1048
Partie 2: 1049
Partie 3: 1048
Partie 4: 1049


#### DATAFRAME 

In [44]:
%%time 
df_movie_info1=create_movie_list(quart)

CPU times: user 3min 37s, sys: 17.1 s, total: 3min 54s
Wall time: 21min 6s


In [45]:
%%time
df_movie_info2=create_movie_list(deux_quart)

CPU times: user 3min 54s, sys: 20.2 s, total: 4min 14s
Wall time: 21min 41s


In [46]:
%%time
df_movie_info3=create_movie_list(trois_quart)

CPU times: user 3min 52s, sys: 19.9 s, total: 4min 12s
Wall time: 20min 36s


In [46]:
%%time
df_movie_info4=create_movie_list(quatre_quart)

CPU times: user 3min 52s, sys: 19.9 s, total: 4min 12s
Wall time: 20min 36s


In [56]:
#passage de chaque dataframe intermediaire en csv pour les sauvgarder

dataframes = [df_movie_info1, df_movie_info2, df_movie_info3, df_movie_info4]

for i, df in enumerate(dataframes, start=1):
    nom_fichier_csv = f'df_movie_info{i}.csv'
    df.to_csv(nom_fichier_csv, index=False)

In [58]:
# Rassembler les DataFrames en un seul DataFrame
df_tmdb = pd.concat(dataframes, ignore_index=True)

# Sauvegarder le DataFrame tmdb en CSV
df_tmdb.to_csv('df_tmdb.csv', index=False)

# Afficher le DataFrame final
df_tmdb.head()

,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,...,spoken_languages,status,tagline,title,video,vote_average,vote_count,success,status_code,status_message
0,False,/ecvy2kMxsJ60ej52beZ0F8EOGkL.jpg,None,80000000.0,"Drama, Thriller, Mystery",http://www.shutterisland.com/,11324.0,tt1130884,en,Shutter Island,...,"English, Deutsch",Released,Some places never let you go.,Shutter Island,False,8.201,22634.0,NaN,NaN,NaN
1,False,None,None,0.0,Documentary,,1157141.0,tt5445282,en,Shutter Island: Into the Lighthouse,...,English,Released,,Shutter Island: Into the Lighthouse,False,0.000,0.0,NaN,NaN,NaN
2,False,/8ZTVqvKDQ8emSGUEMjsS4yHAwrp.jpg,None,160000000.0,"Action, Science Fiction, Adventure",https://www.warnerbros.com/movies/inception,27205.0,tt1375666,en,Inception,...,"English, Français, 日本語, Kiswahili",Released,Your mind is the scene of the crime.,Inception,False,8.366,34886.0,NaN,NaN,NaN
3,False,/JeGkRdNsOuMrgwBdtB0hp763MU.jpg,El Crack Collection,0.0,"Drama, Thriller",https://filmaxinternationalsales.com/film/the-...,613092.0,tt6793710,es,El crack cero,...,Español,Released,The prequel to the cult film El Crack,The Crack: Inception,False,6.700,36.0,NaN,NaN,NaN
4,False,None,None,0.0,"Animation, Action, Thriller, Science Fiction",,64956.0,tt5295894,en,Inception: The Cobol Job,...,English,Released,,Inception: The Cobol Job,False,7.300,290.0,NaN,NaN,NaN


In [1]:
import pandas as pd 

In [ ]:
https://drive.google.com/file/d/1Qt19I-ZxyIc5RU6NktV2k-3gRi-j6KhD/view?usp=drive_link

In [2]:
# Remplacez 'YOUR_FILE_ID' par l'ID de votre fichier CSV
file_id = '1Qt19I-ZxyIc5RU6NktV2k-3gRi-j6KhD'

# Lien de partage direct du fichier CSV sur Google Drive
url = f'https://drive.google.com/uc?id={file_id}'

# Charger le fichier CSV dans un DataFrame sans le télécharger localement
df = pd.read_csv(url)

# Maintenant, vous pouvez manipuler les données dans df
print(df.head())


Empty DataFrame
Columns: [<!DOCTYPE html><html><head><title>Google Drive - Virus scan warning</title><meta http-equiv="content-type" content="text/html; charset=utf-8"/><style nonce="TAxm4O3c7j8iA50N-ak7zg">.goog-link-button{position:relative;color:#15c;text-decoration:underline;cursor:pointer}.goog-link-button-disabled{color:#ccc;text-decoration:none;cursor:default}body{color:#222;font:normal 13px/1.4 arial, sans-serif;margin:0}.grecaptcha-badge{visibility:hidden}.uc-main{padding-top:50px;text-align:center}#uc-dl-icon{display:inline-block;margin-top:16px;padding-right:1em;vertical-align:top}#uc-text{display:inline-block;max-width:68ex;text-align:left}.uc-error-caption, .uc-warning-caption{color:#222;font-size:16px}#uc-download-link{text-decoration:none}.uc-name-size a{color:#15c;text-decoration:none}.uc-name-size a:visited{color:#61c;text-decoration:none}.uc-name-size a:active{color:#d14836;text-decoration:none}.uc-footer{color:#777;font-size:11px;padding-bottom:5ex;padding-top:5ex;te

In [4]:
import pandas as pd

# Remplacez l'ID du fichier dans l'URL ci-dessous
file_id = '1Qt19I-ZxyIc5RU6NktV2k-3gRi-j6KhD'
url = f'https://drive.google.com/uc?export=download&id={file_id}'

# Charger le fichier CSV dans un DataFrame
df = pd.read_csv(url)

# Maintenant, vous pouvez manipuler les données dans df
print(df.head())

Empty DataFrame
Columns: [<!DOCTYPE html><html><head><title>Google Drive - Virus scan warning</title><meta http-equiv="content-type" content="text/html; charset=utf-8"/><style nonce="PrVNZYDuWsNh8Bp58Q74Vw">.goog-link-button{position:relative;color:#15c;text-decoration:underline;cursor:pointer}.goog-link-button-disabled{color:#ccc;text-decoration:none;cursor:default}body{color:#222;font:normal 13px/1.4 arial, sans-serif;margin:0}.grecaptcha-badge{visibility:hidden}.uc-main{padding-top:50px;text-align:center}#uc-dl-icon{display:inline-block;margin-top:16px;padding-right:1em;vertical-align:top}#uc-text{display:inline-block;max-width:68ex;text-align:left}.uc-error-caption, .uc-warning-caption{color:#222;font-size:16px}#uc-download-link{text-decoration:none}.uc-name-size a{color:#15c;text-decoration:none}.uc-name-size a:visited{color:#61c;text-decoration:none}.uc-name-size a:active{color:#d14836;text-decoration:none}.uc-footer{color:#777;font-size:11px;padding-bottom:5ex;padding-top:5ex;te

In [ ]:
%%time
df_film_ac_clean_3 = create_movie_list(liste_films_ac_3)
# CPU times: user 3min 18s, sys: 18.3 s, total: 3min 36s
# Wall time: 20min 22s

CPU times: user 3min 18s, sys: 18.3 s, total: 3min 36s
Wall time: 20min 22s


In [5]:
import pandas as pd

# Remplacez 'YOUR_DROPBOX_LINK' par le lien Dropbox direct de votre fichier
dropbox_link = 'https://www.dropbox.com/scl/fi/0y7md4elgrv6sh57tw9rv/df_critiques_modif.csv?rlkey=tzzcl0jc53lm6znje5379tkgu&dl=1'

# Charger le fichier CSV dans un DataFrame
df = pd.read_csv(dropbox_link)

# Maintenant, vous pouvez manipuler les données dans df
print(df.head())

   id_allocine                                           Critique  \
0       239331  "10.000 days" est à la base une série créée pa...   
1       239331  Les effets spéciaux bas de gamme mettent les a...   
2       239331  10,000 somnifères. A peine ça commence qu'un g...   
3       239331  "10 000 days" est d'une nullité abyssale. Tout...   
4       239331  Beaucoup de parlotes, pas d'action ni d'effets...   

   Note_critique_float    new_date  Polarité_réelle  nb_mots  \
0                  0.5  2015-10-05                0      129   
1                  1.0  2015-12-12                0       18   
2                  1.0  2017-01-10                0       45   
3                  0.5  2016-05-24                0       84   
4                  1.5  2017-01-01                0       22   

                                    Critique_nettoye  \
0  day base série créée eric small réalise film o...   
1  effets spéciaux ba gamme mettent acteurs situa...   
2  somnifères a peine ça commenc

In [6]:
df.head()

,id_allocine,Critique,Note_critique_float,new_date,Polarité_réelle,nb_mots,Critique_nettoye,Text_blob_critique_nettoyé
0,239331,"""10.000 days"" est à la base une série créée pa...",0.5,2015-10-05,0,129,day base série créée eric small réalise film o...,0
1,239331,Les effets spéciaux bas de gamme mettent les a...,1.0,2015-12-12,0,18,effets spéciaux ba gamme mettent acteurs situa...,0
2,239331,"10,000 somnifères. A peine ça commence qu'un g...",1.0,2017-01-10,0,45,somnifères a peine ça commence gar raconte lif...,0
3,239331,"""10 000 days"" est d'une nullité abyssale. Tout...",0.5,2016-05-24,0,84,10 000 day nullité abyssale tout concourt fair...,0
4,239331,"Beaucoup de parlotes, pas d'action ni d'effets...",1.5,2017-01-01,0,22,beaucoup parlotes ni spéciaux histoire molle t...,0
